In [1]:
import os
import glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import h5py
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from monai.transforms import Compose, ScaleIntensity, EnsureType
from tqdm import tqdm

# --- CONFIG ---
BATCH_SIZE = 16 # Diffusion needs smaller batches usually due to VRAM
LR = 1e-4
MAX_EPOCHS = 50
TIMESTEPS = 1000  # Diffusion steps
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_DIR = r"C:\Users\shara\.cache\kagglehub\datasets\awsaf49\brats2020-training-data\versions\3\BraTS2020_training_data\content\data"
META_FILE = r"C:\Users\shara\.cache\kagglehub\datasets\awsaf49\brats2020-training-data\versions\3\BraTS2020_training_data\content\data\meta_data.csv"

In [2]:
print("⏳ Preparing Dataframes...")
df = pd.read_csv(META_FILE)

# 1. Setup Paths
df['filename'] = df['slice_path'].apply(lambda x: os.path.basename(x))
df['local_path'] = df['filename'].apply(lambda x: os.path.join(DATA_DIR, x))

# 2. Extract Slice Number (Assumes filename format like '...slice_123.h5')
# Adjust the split logic if your filenames are different!
df['slice_num'] = df['filename'].apply(lambda x: int(x.split('_')[-1].split('.')[0]))

# 3. Create a Lookup Dictionary: (Volume_ID, Slice_Num) -> File_Path
# This allows us to find neighbors even if the dataset is shuffled
SLICE_MAP = df.set_index(['volume', 'slice_num'])['local_path'].to_dict()

# 4. Splitting & Balancing (Your original logic)
unique_patients = df['volume'].unique()[:100]
train_ids, val_ids = train_test_split(unique_patients, test_size=0.2, random_state=42)

train_df_raw = df[df['volume'].isin(train_ids)]
val_df = df[df['volume'].isin(val_ids)]

# Balance Tumor/Healthy
df_tumor = train_df_raw[train_df_raw['target'] == 1]
df_healthy = train_df_raw[train_df_raw['target'] == 0]
df_healthy_balanced = df_healthy.sample(n=len(df_tumor), random_state=42)

train_df = pd.concat([df_tumor, df_healthy_balanced])
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✅ Data Ready: Train Size {len(train_df)} | Val Size {len(val_df)}")

⏳ Preparing Dataframes...
✅ Data Ready: Train Size 10658 | Val Size 3100


In [ ]:
class BraTS25DDiffusionDataset(Dataset):
    def __init__(self, dataframe, slice_map, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.slice_map = slice_map
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def load_file(self, path):
        """Helper to safely load one H5 file"""
        if path is None: return None # Handle missing neighbors
        try:
            with h5py.File(path, 'r') as f:
                img = f['image'][:] 
                mask = f['mask'][:]
            return img, mask
        except:
            return None, None

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        vol_id = row['volume']
        center_slice = row['slice_num']
        
        # 1. Identify Paths for [Previous, Center, Next]
        slice_indices = [center_slice - 1, center_slice, center_slice + 1]
        stack_imgs = []
        
        # 2. Load the 2.5D Stack
        for s_idx in slice_indices:
            path = self.slice_map.get((vol_id, s_idx))
            
            # If neighbor doesn't exist (edge case), use the center slice again (padding)
            if path is None: 
                path = self.slice_map.get((vol_id, center_slice))
            
            img, _ = self.load_file(path) # We don't need neighbor masks, only images
            
            # Fix Shape: (H, W, 4) -> (4, H, W)
            if img is not None:
                if img.ndim == 3 and img.shape[-1] == 4:
                    img = np.transpose(img, (2, 0, 1))
                stack_imgs.append(torch.tensor(img, dtype=torch.float32))
            else:
                # Fallback if file corrupt
                return self.__getitem__((idx + 1) % len(self.dataframe))

        # Stack shape: [3 (slices), 4 (modalities), H, W] -> Flatten to [12, H, W]
        # This is the "Condition" (MRI Context)
        condition_stack = torch.cat(stack_imgs, dim=0) 

        # 3. Load Target Mask (Center Slice Only)
        _, mask = self.load_file(self.slice_map.get((vol_id, center_slice)))
        if mask.ndim == 3: mask = mask[:, :, 0]
        wt = (mask > 0).astype(np.float32)
        wt = (wt * 2) - 1
        target_mask = torch.tensor(wt[None, :, :], dtype=torch.float32) # [1, H, W]

        # Apply transforms if needed (apply same transform to stack and mask manually if doing spatial augs)
        if self.transform:
            condition_stack = self.transform(condition_stack)
            # Ensure mask isn't scaled weirdly by transforms intended for images
            
        return condition_stack, target_mask

In [4]:
class DiffusionUNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Input = 1 (Noisy Mask) + 12 (3 slices * 4 modalities) = 13 Channels
        self.inc = nn.Sequential(
            nn.Conv2d(13, 64, 3, padding=1),
            nn.GroupNorm(8, 64),
            nn.ReLU()
        )
        # x1 output: 64 channels
        
        self.down1 = nn.Sequential(nn.MaxPool2d(2), nn.Conv2d(64, 128, 3, padding=1), nn.GroupNorm(8, 128), nn.ReLU())
        # x2 output: 128 channels
        
        self.down2 = nn.Sequential(nn.MaxPool2d(2), nn.Conv2d(128, 256, 3, padding=1), nn.GroupNorm(8, 256), nn.ReLU())
        # x3 output: 256 channels (passed to bottleneck)
        
        self.bot = nn.Sequential(nn.Conv2d(256, 512, 3, padding=1), nn.GroupNorm(8, 512), nn.ReLU())

        # --- DECODER ---

        # UP 1
        self.up1 = nn.ConvTranspose2d(512, 256, 2, stride=2) 
        # up1 output: 256. 
        # Concatenates with x2 (128). Total = 256 + 128 = 384.
        
        # FIX 1: Input channels changed from 512 -> 384
        self.up1_conv = nn.Sequential(
            nn.Conv2d(384, 256, 3, padding=1), 
            nn.GroupNorm(8, 256), 
            nn.ReLU()
        ) 
        
        # UP 2
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        # up2 output: 128.
        # Concatenates with x1 (64). Total = 128 + 64 = 192.
        
        # FIX 2: Input channels changed from 256 -> 192
        self.up2_conv = nn.Sequential(
            nn.Conv2d(192, 128, 3, padding=1), 
            nn.GroupNorm(8, 128), 
            nn.ReLU()
        )

        self.out = nn.Conv2d(128, 1, 1)

        # Time Embedding
        self.time_mlp = nn.Sequential(
            nn.Linear(1, 128),
            nn.SiLU(),
            nn.Linear(128, 128)
        )

    def forward(self, x, t, condition):
        # x: Noisy Mask [B, 1, H, W]
        # condition: MRI Stack [B, 12, H, W]
        
        x_input = torch.cat([x, condition], dim=1) # [B, 13, H, W]
        
        # Time processing
        t = t.float().unsqueeze(-1).to(x.device)
        t_emb = self.time_mlp(t).unsqueeze(-1).unsqueeze(-1)
        
        # Encoder
        x1 = self.inc(x_input)         # [B, 64, H, W]
        x2 = self.down1(x1)            # [B, 128, H/2, W/2]
        x3 = self.down2(x2 + t_emb)    # [B, 256, H/4, W/4]  <-- Add time here
        
        # Bottleneck
        b = self.bot(x3)               # [B, 512, H/4, W/4]
        
        # Decoder
        u1 = self.up1(b)               # [B, 256, H/2, W/2]
        
        # Resize if dimensions don't match exactly (e.g. odd input sizes)
        if u1.shape != x2.shape: u1 = F.interpolate(u1, size=x2.shape[2:])
            
        u1 = torch.cat([u1, x2], dim=1)# [B, 256+128=384, H/2, W/2]
        u1 = self.up1_conv(u1)         # [B, 256, H/2, W/2]
        
        u2 = self.up2(u1)              # [B, 128, H, W]
        
        if u2.shape != x1.shape: u2 = F.interpolate(u2, size=x1.shape[2:])
            
        u2 = torch.cat([u2, x1], dim=1)# [B, 128+64=192, H, W]
        u2 = self.up2_conv(u2)         # [B, 128, H, W]
        
        return self.out(u2)

In [ ]:
# --- HELPERS ---
def get_beta_schedule(timesteps):
    return torch.linspace(1e-4, 0.02, timesteps).to(DEVICE)

betas = get_beta_schedule(TIMESTEPS)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)

def forward_diffusion(x_0, t):
    """ Adds noise to the mask """
    noise = torch.randn_like(x_0)
    sqrt_alpha_t = sqrt_alphas_cumprod[t][:, None, None, None]
    sqrt_one_minus_alpha_t = sqrt_one_minus_alphas_cumprod[t][:, None, None, None]
    x_t = sqrt_alpha_t * x_0 + sqrt_one_minus_alpha_t * noise
    return x_t, noise

# --- EXECUTION ---
train_ds = BraTS25DDiffusionDataset(train_df, SLICE_MAP, transform=Compose([ScaleIntensity(), EnsureType()]))
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0) # Workers=0 for H5 safety on Windows

model = DiffusionUNet().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scaler = torch.amp.GradScaler('cuda') # Fixed new syntax for torch > 2.0

print("🚀 Starting Diffusion Training...")

for epoch in range(MAX_EPOCHS):
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{MAX_EPOCHS}")
    
    for batch in progress_bar:
        # MRI Stack [B, 12, H, W] (Condition)
        # Tumor Mask [B, 1, H, W] (Target)
        condition, mask = batch['image'] if isinstance(batch, dict) else batch
        
        condition = condition.to(DEVICE)
        mask = mask.to(DEVICE)
        
        # 1. Sample Random Timesteps
        t = torch.randint(0, TIMESTEPS, (mask.shape[0],), device=DEVICE).long()
        
        # 2. Add Noise to Mask
        noisy_mask, noise = forward_diffusion(mask, t)
        
        # 3. Train
        optimizer.zero_grad()
        
        with torch.amp.autocast('cuda'):
            # Model predicts the NOISE, given the NoisyMask + CleanMRI + Time
            noise_pred = model(noisy_mask, t, condition)
            loss = F.mse_loss(noise, noise_pred) # MSE Loss for Diffusion
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        epoch_loss += loss.item()
        progress_bar.set_description(f"Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1} Avg Loss: {epoch_loss / len(train_loader):.5f}")

# Save Model
torch.save(model.state_dict(), "brats_diffusion_25d.pth")
print("💾 Model Saved.")

🚀 Starting Diffusion Training...


Loss: 0.0017: 100%|██████████| 667/667 [11:07<00:00,  1.00s/it]


Epoch 1 Avg Loss: 0.02264


Loss: 0.0020: 100%|██████████| 667/667 [09:22<00:00,  1.19it/s]


Epoch 2 Avg Loss: 0.00577


Loss: 0.0029:  22%|██▏       | 144/667 [02:14<07:19,  1.19it/s]